In [1]:
import random
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np
import torch.utils.data
import cv2
import torchvision.models.segmentation
import torch
import os
batchSize=1

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [2]:
trainDir="/home/madmax/Documents/SK_articles/final_test/coco_car_test"
#trainDir="/home/madmax/Documents/SK_articles/to_git/valid_data"
imgs=[]
for pth in os.listdir(trainDir):
    imgs.append(trainDir+"/"+pth +"//")
imageSize=[200,200]

In [3]:
def loadData2(idx):
  batch_Imgs=[]
  batch_Data=[]
 #fll 1
 #frl 2
 #bll 3
 #brl 4
  for i in range(batchSize):        
        num_objs = 0
        #idx=0
        #idx=random.randint(0,len(imgs)-1)
        img = cv2.imread(os.path.join(imgs[idx], "original.png"))
        img = cv2.resize(img, imageSize, cv2.INTER_LINEAR)        
        masks=[]
        labels=[]
        maskDir=os.path.join(imgs[idx], "front_left_light")
        isExist = os.path.exists(maskDir)
        if (isExist):
          for mskName in os.listdir(maskDir):
            vesMask = cv2.imread(maskDir+'/'+mskName, 0)
            vesMask = (vesMask > 0).astype(np.uint8) 
            vesMask=cv2.resize(vesMask,imageSize,cv2.INTER_NEAREST)
            masks.append(vesMask)
            labels.append(1)
        
        maskDir=os.path.join(imgs[idx], "front_right_light")
        isExist = os.path.exists(maskDir)
        if (isExist):
          for mskName in os.listdir(maskDir):
            vesMask = cv2.imread(maskDir+'/'+mskName, 0)
            vesMask = (vesMask > 0).astype(np.uint8) 
            vesMask=cv2.resize(vesMask,imageSize,cv2.INTER_NEAREST)
            masks.append(vesMask)     
            labels.append(2)
        maskDir=os.path.join(imgs[idx], "back_left_light")
        isExist = os.path.exists(maskDir)
        if (isExist):
          for mskName in os.listdir(maskDir):
            vesMask = cv2.imread(maskDir+'/'+mskName, 0)
            vesMask = (vesMask > 0).astype(np.uint8) 
            vesMask=cv2.resize(vesMask,imageSize,cv2.INTER_NEAREST)
            masks.append(vesMask)     
            labels.append(3)
        maskDir=os.path.join(imgs[idx], "back_right_light")
        isExist = os.path.exists(maskDir)
        if (isExist):
          for mskName in os.listdir(maskDir):
            vesMask = cv2.imread(maskDir+'/'+mskName, 0)
            vesMask = (vesMask > 0).astype(np.uint8) 
            vesMask=cv2.resize(vesMask,imageSize,cv2.INTER_NEAREST)
            masks.append(vesMask)     
            labels.append(4)
            
        
        
        #all masks
        num_objs = len(masks)
        if num_objs==0: return loadData()        
        boxes = torch.zeros([num_objs,4], dtype=torch.float32)
        for i in range(num_objs):
            x,y,w,h = cv2.boundingRect(masks[i])
            boxes[i] = torch.tensor([x, y, x+w, y+h])
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        img = torch.as_tensor(img, dtype=torch.float32)        
        #all data
        data = {}
        data["boxes"] =  boxes
        data["labels"] =  torch.as_tensor(labels, dtype=torch.int64)   
        #data["labels"] =  torch.ones((num_objs,), dtype=torch.int64)   
        data["masks"] = masks        
        batch_Imgs.append(img)
        batch_Data.append(data)  
  
  batch_Imgs=torch.stack([torch.as_tensor(d) for d in batch_Imgs],0)
  batch_Imgs = batch_Imgs.swapaxes(1, 3).swapaxes(2, 3)
  return batch_Imgs, batch_Data

In [4]:
#test/home/madmax/Documents/SK_articles/coco_to_masks
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')  
model=torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True) 
in_features = model.roi_heads.box_predictor.cls_score.in_features 
model.roi_heads.box_predictor=FastRCNNPredictor(in_features,num_classes=5)
model.load_state_dict(torch.load("7000.torch"))
model.to(device)# move model to the right devic
model.eval()

/home/madmax/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/madmax/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [5]:
 #fll 1
 #frl 2
 #bll 3
 #brl 4
classes = ['_background','front left', 'front right', 'back left', 'back right']


In [6]:
#a = [[0,0,0]] * 10
#data for iou
id1 = 0
print(len(imgs))
images, targets = loadData2(id1)
images = list(image.to(device) for image in images)
targets=[{k: v.to(device) for k,v in t.items()} for t in targets]



def make_predicting_buf(pred, score, imageSize):
    result_predicting = [[0,0,0,0,0]] * imageSize[0] * imageSize[1]
    
    for i in range(len(pred[0]['masks'])):
        msk=pred[0]['masks'][i,0].detach().cpu().numpy()
        scr=pred[0]['scores'][i].detach().cpu().numpy()
        label = pred[0]['labels'][i].detach().cpu().numpy()
        if scr>score:  
            
            counter = 0
            for i1 in range(len(msk)):
                for j in range(len(msk[0])):
                  #print(msk[i1][j] )
                  if (msk[i1][j] > 0.5):
                    buf = [0,0,0,0,0]
                    buf[label] = 1
                    result_predicting[counter] = buf
                    
                  counter+=1
    ypred = torch.tensor(result_predicting)
    return (ypred)



def make_target_buf(targets, imageSize):
    result_targeting = [0] * imageSize[0] * imageSize[1]
    for i in range(len(targets[0]['masks'])):
            msk=targets[0]['masks'][i].detach().cpu().numpy()
            label = targets[0]['labels'][i].detach().cpu().numpy()
            counter = 0
            for i1 in range(len(msk)):
                    for j in range(len(msk[0])):
                      #print(msk[i1][j] )
                      if (msk[i1][j] > 0.5):
                        result_targeting[counter] = label

                      counter+=1
    ytrue = torch.tensor(result_targeting)
    return (ytrue)

    
print(targets[0]['labels'])    
pred = model(images)
    


yy_pred = make_predicting_buf(pred,0.5, imageSize)
counter = 0
yy_true = make_target_buf(targets, imageSize)
print()






def get_list_of_unique(yy_true):
    unique_list = [0,0,0,0,0]
    unique = set(yy_true.tolist())
    for i in unique:
        unique_list[i] = 1
    return unique_list
    
a = get_list_of_unique(yy_true)
b = get_list_of_unique(yy_true)
sum = []
for value1, value2 in zip(a, b):
    sum.append(value1 + value2)
sum



57
tensor([3, 4], device='cuda:0')


/tmp/ipykernel_19857/2327945959.py:63: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  masks = torch.as_tensor(masks, dtype=torch.uint8)


[2, 0, 0, 2, 2]

In [7]:
import ignite

from collections import OrderedDict

import torch
from torch import nn, optim

from ignite.engine import *
from ignite.handlers import *
from ignite.metrics import *
from ignite.utils import *
from ignite.contrib.metrics.regression import *
from ignite.contrib.metrics import *

# create default evaluator for doctests

def eval_step(engine, batch):
    return batch

default_evaluator = Engine(eval_step)
default_evaluator2 = Engine(eval_step)

cm = ConfusionMatrix(num_classes=5)
metric = IoU(cm, ignore_index=0)
metric.attach(default_evaluator, 'iou')
state = default_evaluator.run([[ yy_pred, yy_true]])
print(state.metrics['iou'])





tensor([0.0000, 0.0000, 0.8770, 0.8746], dtype=torch.float64)


In [8]:
def saver_predict_with_classes (images, network, path_out, score, id1): 
     images = cv2.resize(images, imageSize, cv2.INTER_LINEAR)
     images = torch.as_tensor(images, dtype=torch.float32).unsqueeze(0)
     images=images.swapaxes(1, 3).swapaxes(2, 3)
     images = list(image.to(device) for image in images)
     im= images[0].swapaxes(0, 2).swapaxes(0, 1).detach().cpu().numpy().astype(np.uint8)
     im2 = im.copy()
     class_color_list = [[0,0,0],[0,0,128],[0,0,255], [255, 0,0], [255,0,128]]
     with torch.no_grad():
       pred = network(images)
     print(pred[0]['labels'])
     for i in range(len(pred[0]['masks'])):
        msk=pred[0]['masks'][i,0].detach().cpu().numpy()
        scr=pred[0]['scores'][i].detach().cpu().numpy()
        label = pred[0]['labels'][i].detach().cpu().numpy()
        if scr>score:
            im2[:,:,0][msk>0.5], im2[:,:,1][msk>0.5], im2[:,:,2][msk>0.5] = class_color_list[label]        
     print(path_out+str(id1)+'.png')
     cv2.imwrite(path_out+str(id1)+'.png', np.hstack([im,im2]))

                        

In [9]:
path_out = '/home/madmax/Documents/SK_articles/final_test/output_final/'

img = cv2.imread(os.path.join(imgs[id1], "original.png"))
saver_predict_with_classes(img, model, path_out, 0.5, id1)
#images[0].tolist()

tensor([3, 4, 3, 4], device='cuda:0')
/home/madmax/Documents/SK_articles/final_test/output_final/0.png


In [10]:
# full iou
full_iou = True
list_of_unique = []
if full_iou:
    
    out = 0
    for i in range(len(imgs)):

        images, targets = loadData2(i)
        images = list(image.to(device) for image in images)
        targets=[{k: v.to(device) for k,v in t.items()} for t in targets]
        pred = model(images)  
        yy_pred = make_predicting_buf(pred,0.5, imageSize)
        counter = 0
        yy_true = make_target_buf(targets, imageSize)
        cm = ConfusionMatrix(num_classes=5)
        #metric = IoU(cm, ignore_index=0)
        metric = IoU(cm)
        metric.attach(default_evaluator, 'iou')
        state = default_evaluator.run([[ yy_pred, yy_true]])
        if (i==0):
            out = state.metrics['iou']
            list_of_unique = get_list_of_unique(yy_true)
        else: 
            out += state.metrics['iou']
            buf_list_of_unique = get_list_of_unique(yy_true)
            sum = []
            for value1, value2 in zip(list_of_unique, buf_list_of_unique):
                sum.append(value1 + value2)
            list_of_unique = sum
            
        print(state.metrics['iou'])
        img = cv2.imread(os.path.join(imgs[i], "original.png"))
        saver_predict_with_classes(img, model, path_out, 0.5, i)
    print("final IOU:    " )
    print(out/len(imgs))
    print('all_right')
    out_list = []
    print(list_of_unique)
    for value1, value2 in zip(out.tolist(), list_of_unique):
                out_list.append(value1 / value2)
    print(out_list)


tensor([0.9950, 0.0000, 0.0000, 0.8770, 0.8746], dtype=torch.float64)
tensor([3, 4, 3, 4], device='cuda:0')
/home/madmax/Documents/SK_articles/final_test/output_final/0.png
tensor([0.9971, 0.9223, 0.8815, 0.0000, 0.0000], dtype=torch.float64)
tensor([2, 1], device='cuda:0')
/home/madmax/Documents/SK_articles/final_test/output_final/1.png
tensor([0.9948, 0.0000, 0.0000, 0.8701, 0.9199], dtype=torch.float64)
tensor([3, 4], device='cuda:0')
/home/madmax/Documents/SK_articles/final_test/output_final/2.png
tensor([0.9944, 0.0000, 0.0000, 0.8590, 0.9030], dtype=torch.float64)
tensor([3, 4], device='cuda:0')
/home/madmax/Documents/SK_articles/final_test/output_final/3.png
tensor([0.9972, 0.0000, 0.0000, 0.9394, 0.7738], dtype=torch.float64)
tensor([3, 4], device='cuda:0')
/home/madmax/Documents/SK_articles/final_test/output_final/4.png
tensor([0.9882, 0.0000, 0.0000, 0.7423, 0.6737], dtype=torch.float64)
tensor([4, 3], device='cuda:0')
/home/madmax/Documents/SK_articles/final_test/output_fina

In [11]:
#print(out/len(imgs))

In [12]:
#tester
img = cv2.imread("/home/madmax/Documents/SK_articles/final_test/images/image0.png")
saver_predict_with_classes(img, model, path_out, 0.5, 1122)

tensor([2, 1], device='cuda:0')
/home/madmax/Documents/SK_articles/final_test/output_final/1122.png


In [18]:
#segmentation iou
#a = [[0,0,0]] * 10
#data for iou
id1 = 0
#print(len(imgs))
images, targets = loadData2(id1)
images = list(image.to(device) for image in images)
targets=[{k: v.to(device) for k,v in t.items()} for t in targets]



def make_predicting_buf1(pred, score, imageSize, classes_flag):
    if classes_flag:
      result_predicting = [[0,0,0,0,0]] * imageSize[0] * imageSize[1]
    else:
      result_predicting = [[0,0]] * imageSize[0] * imageSize[1]
    
    for i in range(len(pred[0]['masks'])):
        msk=pred[0]['masks'][i,0].detach().cpu().numpy()
        scr=pred[0]['scores'][i].detach().cpu().numpy()
        label = pred[0]['labels'][i].detach().cpu().numpy()
        if scr>score:              
            counter = 0
            for i1 in range(len(msk)):
                for j in range(len(msk[0])):
                  if (msk[i1][j] > 0.5):
                    if classes_flag:
                      buf = [0,0,0,0,0]
                      buf[label] = 1
                    else:
                       buf = [0,0]
                       buf[1] = 1
                    result_predicting[counter] = buf
                    
                  counter+=1
    ypred = torch.tensor(result_predicting)
    return (ypred)



def make_target_buf1(targets, imageSize, classes_flag):
    result_targeting = [0] * imageSize[0] * imageSize[1]
    for i in range(len(targets[0]['masks'])):
            msk=targets[0]['masks'][i].detach().cpu().numpy()
            label = targets[0]['labels'][i].detach().cpu().numpy()
            counter = 0
            for i1 in range(len(msk)):
                    for j in range(len(msk[0])):
                      #print(msk[i1][j] )
                      if (msk[i1][j] > 0.5):
                            if classes_flag:
                                result_targeting[counter] = label
                            else:
                                result_targeting[counter] = 1

                      counter+=1
    ytrue = torch.tensor(result_targeting)
    return (ytrue)

    
print(targets[0]['labels'])    
pred = model(images)
    


yy_pred = make_predicting_buf1(pred,0.5, imageSize, False)
counter = 0
yy_true = make_target_buf1(targets, imageSize, False)
print(yy_pred.shape)
print(yy_true.shape)

cm2 = ConfusionMatrix(num_classes=2)
metric = IoU(cm2, ignore_index=0)
metric.attach(default_evaluator2, 'iou')
state = default_evaluator2.run([[yy_pred, yy_true]])
state


tensor([3, 4], device='cuda:0')
torch.Size([40000, 2])
torch.Size([40000])


State:
	iteration: 1
	epoch: 1
	epoch_length: 1
	max_epochs: 1
	output: <class 'list'>
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'list'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

In [19]:
out = []
for i in range(len(imgs)):
    
    images, targets = loadData2(i)
    images = list(image.to(device) for image in images)
    targets=[{k: v.to(device) for k,v in t.items()} for t in targets]
    pred = model(images)  
    yy_pred1 = make_predicting_buf1(pred,0.5, imageSize, False)
    counter = 0
    yy_true1 = make_target_buf1(targets, imageSize, False)
    
    print(yy_pred1.shape)
    print(yy_true1.shape)
    cm2 = ConfusionMatrix(num_classes=2)
    metric = IoU(cm2, ignore_index=0)
    metric.attach(default_evaluator2, 'iou')
    state = default_evaluator2.run([[ yy_pred1, yy_true1]])
    
    if (i==0):
        out = state.metrics['iou']
    else: 
        out += state.metrics['iou']
    print(state.metrics['iou'])
    img = cv2.imread(os.path.join(imgs[i], "original.png"))
    saver_predict_with_classes(img, model, path_out, 0.5, i)
print("final segmentation IOU:    " )
print(out/len(imgs))


torch.Size([40000, 2])
torch.Size([40000])
tensor([0.8760], dtype=torch.float64)
tensor([3, 4, 3, 4], device='cuda:0')
/home/madmax/Documents/SK_articles/final_test/output_final/0.png
torch.Size([40000, 2])
torch.Size([40000])
tensor([0.9075], dtype=torch.float64)
tensor([2, 1], device='cuda:0')
/home/madmax/Documents/SK_articles/final_test/output_final/1.png
torch.Size([40000, 2])
torch.Size([40000])
tensor([0.8938], dtype=torch.float64)
tensor([3, 4], device='cuda:0')
/home/madmax/Documents/SK_articles/final_test/output_final/2.png
torch.Size([40000, 2])
torch.Size([40000])
tensor([0.8804], dtype=torch.float64)
tensor([3, 4], device='cuda:0')
/home/madmax/Documents/SK_articles/final_test/output_final/3.png
torch.Size([40000, 2])
torch.Size([40000])
tensor([0.9023], dtype=torch.float64)
tensor([3, 4], device='cuda:0')
/home/madmax/Documents/SK_articles/final_test/output_final/4.png
torch.Size([40000, 2])
torch.Size([40000])
tensor([0.7114], dtype=torch.float64)
tensor([4, 3], device='

In [14]:
#f1 score
import torch
from torchmetrics.functional.classification import multiclass_f1_score
input = torch.tensor([[0.9, 0.1, 0, 0,1], [0.1, 0.2, 0.4, 0.3,0], [0, 1.0, 0, 0,1], [0, 0, 0.2, 0.8,0]])
target = torch.tensor([0, 1, 2, 3])
print(input.size())
print(target.size())


multiclass_f1_score(input, target, num_classes=5, ignore_index = 0)
def make_predicting_buf(pred, score, imageSize):
    result_predicting = [[0,0,0,0,0]] * imageSize[0] * imageSize[1]
    
    for i in range(len(pred[0]['masks'])):
        msk=pred[0]['masks'][i,0].detach().cpu().numpy()
        scr=pred[0]['scores'][i].detach().cpu().numpy()
        label = pred[0]['labels'][i].detach().cpu().numpy()
        if scr>score:  
            
            counter = 0
            for i1 in range(len(msk)):
                for j in range(len(msk[0])):
                  #print(msk[i1][j] )
                  if (msk[i1][j] > 0.5):
                    buf = [0,0,0,0,0]
                    buf[label] = 1
                    result_predicting[counter] = buf
                    
                  counter+=1
    ypred = torch.tensor(result_predicting)
    return (ypred)



def make_target_buf(targets, imageSize):
    result_targeting = [0] * imageSize[0] * imageSize[1]
    for i in range(len(targets[0]['masks'])):
            msk=targets[0]['masks'][i].detach().cpu().numpy()
            label = targets[0]['labels'][i].detach().cpu().numpy()
            counter = 0
            for i1 in range(len(msk)):
                    for j in range(len(msk[0])):
                      #print(msk[i1][j] )
                      if (msk[i1][j] > 0.5):
                        result_targeting[counter] = label

                      counter+=1
    ytrue = torch.tensor(result_targeting)
    return (ytrue)





images, targets = loadData2(0)
images = list(image.to(device) for image in images)
targets=[{k: v.to(device) for k,v in t.items()} for t in targets]
pred = model(images)  
yy_pred = make_predicting_buf(pred,0.5, imageSize)
counter = 0
print(yy_pred.size())
print(yy_true.size())
yy_pred = yy_pred.type(torch.FloatTensor)

yy_true = make_target_buf(targets, imageSize)
state = multiclass_f1_score(yy_pred, yy_true, num_classes=5, ignore_index=0)
print(state)



torch.Size([4, 5])
torch.Size([4])
torch.Size([40000, 5])
torch.Size([40000])
tensor(0.3860)


In [16]:
f1 = True
if f1:
    
    out = 0
    for i in range(len(imgs)):

        images, targets = loadData2(i)
        images = list(image.to(device) for image in images)
        targets=[{k: v.to(device) for k,v in t.items()} for t in targets]
        pred = model(images)  
        yy_pred = make_predicting_buf(pred,0.5, imageSize)
        yy_pred = yy_pred.type(torch.FloatTensor)
        counter = 0
        yy_true = make_target_buf(targets, imageSize)
        state = multiclass_f1_score(yy_pred, yy_true, num_classes=5)
        print("state", state.float())
        if (i==0):
            out = state.float()
        else: 
            out += state.float()
        print(out)
        #img = cv2.imread(os.path.join(imgs[i], "original.png"))
        #saver_predict_with_classes(img, model, path_out, 0.5, i)
    print("final F1:    " )
    print(len(imgs))    
    print(out/len(imgs))

state tensor(0.5730)
tensor(0.5730)
state tensor(0.5790)
tensor(1.1520)
state tensor(0.5772)
tensor(1.7293)
state tensor(0.5741)
tensor(2.3034)
state tensor(0.5680)
tensor(2.8713)
state tensor(0.5302)
tensor(3.4016)
state tensor(0.1983)
tensor(3.5998)
state tensor(0.5347)
tensor(4.1345)
state tensor(0.5804)
tensor(4.7149)
state tensor(0.1992)
tensor(4.9140)
state tensor(0.1979)
tensor(5.1119)
state tensor(0.5173)
tensor(5.6293)
state tensor(0.1987)
tensor(5.8280)
state tensor(0.4885)
tensor(6.3164)
state tensor(0.5746)
tensor(6.8910)
state tensor(0.1991)
tensor(7.0901)
state tensor(0.5879)
tensor(7.6780)
state tensor(0.5778)
tensor(8.2558)
state tensor(0.5498)
tensor(8.8056)
state tensor(0.3838)
tensor(9.1894)
state tensor(0.3817)
tensor(9.5711)
state tensor(0.2856)
tensor(9.8567)
state tensor(0.1996)
tensor(10.0563)
state tensor(0.5489)
tensor(10.6052)
state tensor(0.3685)
tensor(10.9737)
state tensor(0.5661)
tensor(11.5397)
state tensor(0.5879)
tensor(12.1276)
state tensor(0.3747)
te